In [2]:
%tensorflow_version 2.x

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2022-01-15 07:48:42--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2022-01-15 07:48:42--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc475b16bbbb6772c5d7ab9c4d81.dl.dropboxusercontent.com/zip_download_get/BBP9XWVbRyL4hMcHKYPP8uQZaikHO0K1dmxBZegzA7fT9tD5NdRbTXLzc7ucnceJyl4lNOBhaDdmYAQmk1TUTzhzNuM1GhdnZLV2fdlKWbmvvw# [following]
--2022-01-15 07:48:43--  https://uc475b16bbbb6772c5d7ab9c4d81.dl.dropboxusercontent.com/zip_download_get/BBP9XWVbRyL4hMcHKYPP8uQZaikHO0K1dmxBZegzA7fT9tD5NdRbTXLzc7ucnceJyl4lNOBhaDdmYAQm

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [5]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

In [6]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [7]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [8]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , How can you root a Christmas cactus?                                                                                                                                                                                                          1
How many Hat Man shadow people are there?                                                                                                                                                                                                     1
Generally, would males of non-White descent, such as Chinese or Pakistani, have difficulty dating a woman who wants to know the his race first before dating, such as in the form of a certified DNA test with racial composition results?    1
Does the current political climate make it unwise to suggest men excel in certain areas and women in others due to their choices and not sexism?                                                                                              1
The NY Attorney General 

In [9]:
filter.drop([1306122,1306123],0,inplace=True)

In [10]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [11]:
filter.drop(['PK'],1,inplace=True)

In [12]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [13]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [14]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-01-15 08:01:46--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-01-15 08:01:46--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-01-15 08:01:46--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [15]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [16]:
!rm /content/glove.42B.300d.zip

In [17]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [18]:
len(embeding_index['provide'])

300

In [19]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [20]:
len(embeding_index)

1917494

In [21]:
X=filter['question_text']
Y=filter['target']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [25]:
from sklearn.utils.class_weight import compute_class_weight

In [27]:
x_train

433445         How can you proof that AP is not equal to gp?
638945     How can you form a relationship with a love av...
658833                                  What is a supervote?
209074     Why are most people terrified disproportionate...
202067     Are there stages in between high functioning a...
                                 ...                        
563634     Which is best fund house to invest 1000 rs per...
1030561    Can I work in a hospital in the USA with a uni...
1113401             Will they ever produce edible tide pods?
1031369              What is the expansion for (a+b+c+d) ^4?
776894     Where can one buy strawberry polyphenol extracts?
Name: question_text, Length: 1044897, dtype: object

In [28]:
from nltk import word_tokenize

In [29]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [30]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [31]:
np.percentile(sent_lens,99)

44.0

In [32]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [33]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'was': 48,
 'did': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [34]:
vocab_size

195923

In [35]:
seq_train_matrix.shape

(1044897, 44)

In [36]:
seq_train

[[9, 15, 14, 1422, 19, 2413, 3, 44, 1246, 5, 13766],
 [9, 15, 14, 389, 4, 245, 21, 4, 143, 16631, 99],
 [2, 3, 4, 88322],
 [16,
  12,
  60,
  37,
  7486,
  15501,
  5,
  1,
  2368,
  16970,
  32,
  1299,
  23311,
  4,
  715,
  14605,
  23,
  4153],
 [12, 39, 4778, 6, 69, 167, 4958, 10, 457, 4958, 11735, 20, 52, 2, 12, 47],
 [13, 211, 614, 548, 6, 3229, 148, 30, 10381, 73, 169, 3229, 3, 6097],
 [9, 11, 8, 706, 741, 2862, 43, 273, 7, 3694, 9, 11, 8, 415, 197, 4406],
 [9, 11, 8, 86, 100, 2909],
 [3, 4452, 21, 122, 213, 4344, 212, 32, 499, 13, 4, 110],
 [226, 1393, 220, 6, 30, 3959, 10466, 3, 1591, 46, 5181, 4959],
 [2, 3, 88323],
 [9, 15, 7276, 24, 4, 887, 906],
 [9, 11, 8, 33, 4, 3006, 404, 8842, 21, 3727, 6, 57, 44, 1, 56, 13, 407, 4727],
 [16, 48, 1891, 26038, 234, 5717, 5, 22551, 7, 1, 14824],
 [2, 3, 1, 635, 1262, 89, 399, 6, 4, 3049],
 [18,
  6251,
  334,
  3,
  88324,
  15,
  8,
  697,
  50,
  847,
  6,
  1220,
  107,
  23,
  157,
  6,
  3655,
  23,
  1113,
  1208],
 [11,
  14,
  4

In [37]:
x_train.shape

(1044897,)

In [38]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [39]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [40]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [41]:
 len(embedding_matrix)

195924

In [42]:
embedding_matrix.shape

(195924, 300)

In [43]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.28931999, -0.26679999, -0.036894  , ..., -0.47092   ,
         0.079963  , -0.098361  ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.057839  ,  0.22705001,  0.12846   , ..., -0.21764   ,
         0.39590999,  0.14568   ]])

In [44]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [45]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           58777200  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [46]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [47]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [48]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [49]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2958 - accuracy: 0.8743
Epoch 00001: val_loss improved from inf to 0.25006, saving model to /content/weights-01-0.2501.h5
1045/1045 [==============================] - 188s 175ms/step - loss: 0.2958 - accuracy: 0.8743 - val_loss: 0.2501 - val_accuracy: 0.9034
Epoch 2/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.8952
Epoch 00002: val_loss did not improve from 0.25006
1045/1045 [==============================] - 186s 178ms/step - loss: 0.2508 - accuracy: 0.8952 - val_loss: 0.2544 - val_accuracy: 0.9061
Epoch 3/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2272 - accuracy: 0.9017
Epoch 00003: val_loss did not improve from 0.25006
1045/1045 [==============================] - 186s 178ms/step - loss: 0.2272 - accuracy: 0.9017 - val_loss: 0.2748 - val_accuracy: 0.8889
Epoch 4/15
1045/1045 [==============================] - ETA: 0s - loss: 0.2072 - accuracy: 0.90

In [51]:
best_model = load_model('/content/weights-04-0.1969.h5')

In [ ]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58823400  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [52]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9638168696473797

In [53]:
best_model.save('./spam_filter_model_quora_questions.h5')